In [ ]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta

driver = webdriver.Chrome(executable_path='./chromedriver')

driver.get("https://www.indeed.com/advanced_search")

driver.implicitly_wait(3)

# search analyst
search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
search_job.send_keys(['Analyst'])

# search location
searchLocation = driver.find_element_by_xpath('//input[@id="where"]')
searchLocation.clear()
searchLocation.send_keys("united states")
# searchLocation.send_keys("ohio")

# set display limit of 50 results per page
display_limit = driver.find_element_by_xpath(
    '//select[@id="limit"]//option[@value="50"]')
display_limit.click()
driver.implicitly_wait(3)

# sort display with date
display_sort = driver.find_element_by_xpath(
    '//select[@id="sort"]//option[@value="date"]')
display_sort.click()
driver.implicitly_wait(3)

# limited to 1 day
result_age = driver.find_element_by_xpath(
    '//select[@id="fromage"]//option[@value="1"]')
result_age.click()

# location
result_age = driver.find_element_by_xpath(
    '//select[@id="radius"]//option[@value="0"]')
result_age.click()

driver.implicitly_wait(3)

# push search button
search_button = driver.find_element_by_xpath('//*[@id="fj"]')
search_button.click()
driver.implicitly_wait(3)

# Get exact search result amount
search_count = driver.find_element_by_xpath('//div[@id="searchCount"]').text

def sub_str(s, start, end):
    return s[s.find(start)+len(start):s.rfind(end)]
num = sub_str(search_count, "of ", " jobs")
pages = math.ceil(int(num.replace(',', ''))/50)

In [57]:
print(datetime.now().strftime("%m_%d_%Y"))
print("total pages: ", pages)

01_11_2021
total pages:  43


In [48]:
%%time

links = []
dates = []

for i in range(0, pages):

    job_card = driver.find_elements_by_xpath(
        '//div[contains(@class,"clickcard")]')
    
    for job in job_card:
        
        date = datetime.now().strftime("%m/%d/%Y")
        dates.append(date)

        links.append(job.find_element_by_xpath(
            './/h2[@class="title"]//a').get_attribute(name="href"))
        

    print("Finish the page: {}.".format(str(i+1)))
    driver.implicitly_wait(3)
    next_page = driver.find_elements_by_xpath("//span[@class='pn']")[-1]
    driver.execute_script("arguments[0].click();", next_page)

Finish the page: 1.
Finish the page: 2.
Finish the page: 3.
Finish the page: 4.
Finish the page: 5.
Finish the page: 6.
Finish the page: 7.
Finish the page: 8.
Finish the page: 9.
Finish the page: 10.
Finish the page: 11.
Finish the page: 12.
Finish the page: 13.
Finish the page: 14.
Finish the page: 15.
Finish the page: 16.
Finish the page: 17.
Finish the page: 18.
Finish the page: 19.
Finish the page: 20.
Finish the page: 21.
Finish the page: 22.
Finish the page: 23.
Finish the page: 24.
Finish the page: 25.
Finish the page: 26.
Finish the page: 27.
Finish the page: 28.
Finish the page: 29.
Finish the page: 30.
Finish the page: 31.
Finish the page: 32.
Finish the page: 33.
Finish the page: 34.
Finish the page: 35.
Finish the page: 36.
Finish the page: 37.
Finish the page: 38.
Finish the page: 39.
Finish the page: 40.
Finish the page: 41.
Finish the page: 42.
Finish the page: 43.
CPU times: user 2.33 s, sys: 136 ms, total: 2.46 s
Wall time: 1min 2s


need some code to pull html based on url; give time dealy.

<!-- # %%time
# # Despcriptions
# for link in links:
#     try:
#         driver.get(link)
#         jd = driver.find_element_by_xpath(
#             '//div[@id="jobDescriptionText"]').text
#     except:
#         jd = "None"

#     descriptions.append(jd) -->

In [53]:
df = pd.DataFrame()
df['Date'] = dates
df['Link'] = links

In [54]:
df.head()

,Date,Link
0,01/11/2021,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,01/11/2021,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,01/11/2021,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,01/11/2021,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4,01/11/2021,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


In [58]:
# save to feather file
filename = datetime.now().strftime("Post_URL_"+"%m_%d_%Y")
df.to_feather("./source/"+filename)